In [1]:
# unzip dataset_letras.zip
!unzip dataset_sintetic_lletres.zip

Streaming output truncated to the last 5000 lines.
 extracting: dataset_letras4/P/P_106.png  
 extracting: dataset_letras4/P/P_107.png  
 extracting: dataset_letras4/P/P_108.png  
 extracting: dataset_letras4/P/P_109.png  
 extracting: dataset_letras4/P/P_11.png  
 extracting: dataset_letras4/P/P_110.png  
 extracting: dataset_letras4/P/P_111.png  
 extracting: dataset_letras4/P/P_112.png  
 extracting: dataset_letras4/P/P_113.png  
 extracting: dataset_letras4/P/P_114.png  
 extracting: dataset_letras4/P/P_115.png  
 extracting: dataset_letras4/P/P_116.png  
 extracting: dataset_letras4/P/P_117.png  
 extracting: dataset_letras4/P/P_118.png  
 extracting: dataset_letras4/P/P_119.png  
 extracting: dataset_letras4/P/P_12.png  
 extracting: dataset_letras4/P/P_120.png  
 extracting: dataset_letras4/P/P_121.png  
 extracting: dataset_letras4/P/P_122.png  
 extracting: dataset_letras4/P/P_123.png  
 extracting: dataset_letras4/P/P_124.png  
 extracting: dataset_letras4/P/P_125.png  
 extr

In [2]:
!pip install WandB
import wandb

## Login
wandb.login(key='a14c6a2ec25620e6e2047f787c8dbe5d7710eaef')

## Inicializa WandB
wandb.init(project="repte1_psiv", entity="andreu-mir")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.5/314.5 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: andreu-mir. Use `wandb login --relogin` to force relogin


In [3]:
import os
import joblib
import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import KFold
import wandb
# Iniciar WandB
wandb.init(project="SVM_CrossValidation_letters")

# Definición de parámetros
batch_size = 32
img_height, img_width = 64, 40  # Tamaño de las imágenes
num_folds = 5  # Número de pliegues (folds) para la validación cruzada

# Crear carpeta 'results' si no existe
results_dir = 'results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),  # Redimensionar las imágenes
    transforms.Grayscale(num_output_channels=1),  # Escala de grises
    transforms.ToTensor(),  # Convertir imágenes a tensores
    transforms.Normalize((0.5,), (0.5,)),  # Normalizar las imágenes
])

# Cargar el dataset
base_dir = 'dataset_letras4'
dataset = datasets.ImageFolder(root=base_dir, transform=transform)

# Función para extraer características planas de las imágenes
def extract_features(loader):
    features = []
    labels = []
    for images, lbls in loader:
        images = images.view(images.size(0), -1)  # Flatten each image to a vector
        features.append(images.numpy())
        labels.append(lbls.numpy())

    return np.vstack(features), np.hstack(labels)

# Configurar el objeto KFold
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Loop a través de los pliegues de validación cruzada
for fold, (train_indices, val_indices) in enumerate(kfold.split(range(len(dataset)))):

    print(f'Fold {fold + 1}/{num_folds}')

    # Crear subconjuntos de entrenamiento y validación
    train_subset = Subset(dataset, train_indices)
    val_subset = Subset(dataset, val_indices)

    # Crear DataLoaders
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # Extraer características de los conjuntos de entrenamiento y validación
    train_features, train_labels = extract_features(train_loader)
    val_features, val_labels = extract_features(val_loader)

    # Definir el modelo SVM con un pipeline de estandarización
    svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0))

    # Entrenar el modelo SVM
    svm_model.fit(train_features, train_labels)

    # Evaluar el modelo en el conjunto de validación
    val_predictions = svm_model.predict(val_features)
    accuracy = accuracy_score(val_labels, val_predictions)
    precision = precision_score(val_labels, val_predictions, average='weighted')
    recall = recall_score(val_labels, val_predictions, average='weighted')
    f1 = f1_score(val_labels, val_predictions, average='weighted')

    # Imprimir resultados del fold
    print(f"Fold {fold + 1} - Accuracy: {accuracy * 100:.2f}%")
    print(f"Fold {fold + 1} - Precision: {precision:.4f}")
    print(f"Fold {fold + 1} - Recall: {recall:.4f}")
    print(f"Fold {fold + 1} - F1 Score: {f1:.4f}")

    # Log de métricas en WandB
    wandb.log({
        f"Fold {fold + 1} Accuracy": accuracy,
        f"Fold {fold + 1} Precision": precision,
        f"Fold {fold + 1} Recall": recall,
        f"Fold {fold + 1} F1 Score": f1,
    })

    # Obtener la matriz de confusión completa
    cm = confusion_matrix(val_labels, val_predictions)
    num_classes = len(dataset.classes)

    # Para cada clase (letra), calculamos TP, FP, FN, TN y los guardamos
    for class_idx in range(num_classes):
        # Verdaderos Positivos (TP): Valor en la diagonal de la matriz para la clase actual
        TP = cm[class_idx, class_idx]

        # Falsos Positivos (FP): Suma de la columna de la clase, excepto el TP
        FP = cm[:, class_idx].sum() - TP

        # Falsos Negativos (FN): Suma de la fila de la clase, excepto el TP
        FN = cm[class_idx, :].sum() - TP

        # Verdaderos Negativos (TN): Todo lo que no está en la fila ni en la columna de la clase
        TN = cm.sum() - (TP + FP + FN)

        # Guardar TP, FP, TN, FN en un archivo dentro de la carpeta 'results'
        file_path = os.path.join(results_dir, f'confusion_matrix_metrics_class_{class_idx}_fold_{fold + 1}.txt')

        with open(file_path, 'w') as f:
            f.write(f'Clase {dataset.classes[class_idx]} (Clase {class_idx}):\n')
            f.write(f'TP: {TP}\n')
            f.write(f'FP: {FP}\n')
            f.write(f'FN: {FN}\n')
            f.write(f'TN: {TN}\n')

        print(f'Clase {dataset.classes[class_idx]} - Fold {fold + 1}:')
        print(f'TP: {TP}, FP: {FP}, FN: {FN}, TN: {TN}')

# Guardar el modelo entrenado en un archivo
model_filename = "svm_model_leters.pkl"
joblib.dump(svm_model, model_filename)
print(f"Modelo guardado como {model_filename}")

# Descargar el modelo (si estás en un entorno de notebook)
wandb.save(model_filename)

# Finalizar el experimento en WandB
wandb.finish()


Fold 1/5
Fold 1 - Accuracy: 86.71%
Fold 1 - Precision: 0.8688
Fold 1 - Recall: 0.8671
Fold 1 - F1 Score: 0.8669
Clase B - Fold 1:
TP: 91, FP: 12, FN: 17, TN: 1980
Clase C - Fold 1:
TP: 95, FP: 17, FN: 6, TN: 1982
Clase D - Fold 1:
TP: 84, FP: 19, FN: 16, TN: 1981
Clase F - Fold 1:
TP: 91, FP: 23, FN: 12, TN: 1974
Clase G - Fold 1:
TP: 86, FP: 21, FN: 13, TN: 1980
Clase H - Fold 1:
TP: 70, FP: 23, FN: 12, TN: 1995
Clase J - Fold 1:
TP: 112, FP: 4, FN: 3, TN: 1981
Clase K - Fold 1:
TP: 85, FP: 17, FN: 22, TN: 1976
Clase L - Fold 1:
TP: 105, FP: 1, FN: 1, TN: 1993
Clase M - Fold 1:
TP: 98, FP: 17, FN: 8, TN: 1977
Clase N - Fold 1:
TP: 89, FP: 14, FN: 20, TN: 1977
Clase P - Fold 1:
TP: 89, FP: 10, FN: 9, TN: 1992
Clase Q - Fold 1:
TP: 71, FP: 17, FN: 27, TN: 1985
Clase R - Fold 1:
TP: 68, FP: 11, FN: 21, TN: 2000
Clase S - Fold 1:
TP: 69, FP: 13, FN: 12, TN: 2006
Clase T - Fold 1:
TP: 87, FP: 10, FN: 11, TN: 1992
Clase V - Fold 1:
TP: 86, FP: 19, FN: 17, TN: 1978
Clase W - Fold 1:
TP: 101,

Fold 1 Accuracy,▁
Fold 1 F1 Score,▁
Fold 1 Precision,▁
Fold 1 Recall,▁
Fold 2 Accuracy,▁
Fold 2 F1 Score,▁
Fold 2 Precision,▁
Fold 2 Recall,▁
Fold 3 Accuracy,▁
Fold 3 F1 Score,▁
Fold 3 Precision,▁


In [5]:
import zipfile
import os

def zip_folder(folder_path, output_zip):
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Recorrer todos los archivos y carpetas dentro de folder_path
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Escribir cada archivo al ZIP usando un path relativo
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Ruta de la carpeta que quieres comprimir
folder_to_zip = 'svm_model_leters'

# Nombre del archivo zip de salida
output_zip_file = 'svm_model_leters.zip'

# Llamar a la función para crear el zip
zip_folder(folder_to_zip, output_zip_file)

print(f"Carpeta '{folder_to_zip}' comprimida en '{output_zip_file}'")


Carpeta 'svm_model_leters' comprimida en 'svm_model_leters.zip'
